In [1]:
!pwd

/users/vutomi/Source_Detection/YOLOs


In [2]:
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np
import os

# Define paths
fits_file = "/users/vutomi/Source_Detection/abell_133_cropped.fits"
catalog_file = "/users/vutomi/Source_Detection/abell_133_cropped.pybdsf.srl.fits"  # FITS file containing the catalog
output_dir = "/users/vutomi/Source_Detection/YOLOs/Data/labels/"
output_file = os.path.join(output_dir, "abell_133_cropped.txt")

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load FITS image file and WCS
try:
    with fits.open(fits_file) as hdul:
        image_header = hdul[0].header
        wcs = WCS(image_header, naxis=2)
        image_shape = hdul[0].data.shape  # Get image dimensions
    print(f"Successfully loaded FITS image from {fits_file}")
except FileNotFoundError:
    print(f"Error: FITS image file not found at {fits_file}. Please verify the path.")
    raise
except Exception as e:
    print(f"Error loading FITS image: {e}")
    raise

# Load FITS catalog file
try:
    with fits.open(catalog_file) as hdul:
        catalog = hdul[1].data  # Assuming the catalog is in HDU 1
    print(f"Successfully loaded catalog from {catalog_file}")
    print(f"Available columns: {catalog.names}")  # Print column names to verify
except FileNotFoundError:
    print(f"Error: Catalog file not found at {catalog_file}. Please verify the path.")
    raise
except IndexError:
    print(f"Error: No table data found in {catalog_file}. Check the HDU index.")
    raise
except Exception as e:
    print(f"Error loading catalog: {e}")
    raise

# Convert RA, Dec to pixel coordinates
try:
    ra_col = "RA"  # Adjust if the column name differs
    dec_col = "DEC"
    x_pixels, y_pixels = wcs.world_to_pixel_values(catalog[ra_col], catalog[dec_col])
except KeyError:
    print(f"Error: Required columns '{ra_col}' or '{dec_col}' not found in catalog. Available columns: {catalog.names}")
    raise

# Estimate bounding box sizes in pixels
try:
    maj_col = "Maj"
    min_col = "Min"
    pixel_scale = abs(image_header["CDELT1"])  # Degrees per pixel
    maj_pixels = catalog[maj_col] / pixel_scale
    min_pixels = catalog[min_col] / pixel_scale
except KeyError:
    print(f"Error: Required columns '{maj_col}' or '{min_col}' not found in catalog. Available columns: {catalog.names}")
    raise

# Normalize labels to YOLO format (all values should be in [0,1])
image_width, image_height = image_shape[-2:]  # Assuming 2D image

yolo_labels = []
for x, y, maj, min_ in zip(x_pixels, y_pixels, maj_pixels, min_pixels):
    # Ensure bounding boxes are within image bounds
    x_center = np.clip(x / image_width, 0, 1)
    y_center = np.clip(y / image_height, 0, 1)
    width = np.clip(maj / image_width, 0, 1)
    height = np.clip(min_ / image_height, 0, 1)

    # Ignore entries with zero or negative width/height
    if width > 0 and height > 0:
        yolo_labels.append(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

# Save labels
try:
    with open(output_file, "w") as f:
        f.write("\n".join(yolo_labels))
    print(f"Labels saved to {output_file}")
except Exception as e:
    print(f"Error saving labels: {e}")
    raise

Successfully loaded FITS image from /users/vutomi/Source_Detection/abell_133_cropped.fits
Successfully loaded catalog from /users/vutomi/Source_Detection/abell_133_cropped.pybdsf.srl.fits
Available columns: ['Source_id', 'Isl_id', 'RA', 'E_RA', 'DEC', 'E_DEC', 'Total_flux', 'E_Total_flux', 'Peak_flux', 'E_Peak_flux', 'RA_max', 'E_RA_max', 'DEC_max', 'E_DEC_max', 'Maj', 'E_Maj', 'Min', 'E_Min', 'PA', 'E_PA', 'Maj_img_plane', 'E_Maj_img_plane', 'Min_img_plane', 'E_Min_img_plane', 'PA_img_plane', 'E_PA_img_plane', 'DC_Maj', 'E_DC_Maj', 'DC_Min', 'E_DC_Min', 'DC_PA', 'E_DC_PA', 'DC_Maj_img_plane', 'E_DC_Maj_img_plane', 'DC_Min_img_plane', 'E_DC_Min_img_plane', 'DC_PA_img_plane', 'E_DC_PA_img_plane', 'Isl_Total_flux', 'E_Isl_Total_flux', 'Isl_rms', 'Isl_mean', 'Resid_Isl_rms', 'Resid_Isl_mean', 'S_Code']
Labels saved to /users/vutomi/Source_Detection/YOLOs/Data/labels/abell_133_cropped.txt


In [2]:
# save as convert_fits_to_png.py
# save as convert_fits_to_png.py
from astropy.io import fits
import numpy as np
from PIL import Image

def fits_to_png(fits_file, output_png):
    hdu = fits.open(fits_file)[0]
    data = hdu.data
    if data.ndim > 2:  # Collapse frequency/stokes axes
        data = np.nanmean(data, axis=(0, 1) if data.ndim == 4 else 0)
    data = np.nan_to_num(data, nan=np.nanmean(data))
    data = (data - np.min(data)) / (np.max(data) - np.min(data)) * 255
    data = data.astype(np.uint8)
    Image.fromarray(data).save(output_png)
    # Save 2D FITS for WCS
    hdu.data = data
    hdu.header['NAXIS'] = 2
    for key in ['NAXIS3', 'NAXIS4', 'CRVAL3', 'CRPIX3', 'CDELT3', 'CTYPE3', 'CRVAL4', 'CRPIX4', 'CDELT4', 'CTYPE4']:
        hdu.header.remove(key, ignore_missing=True)
    hdu.writeto(fits_file.replace('.fits', '_2d.fits'), overwrite=True)

fits_to_png('Abell_141_aFix_pol_I_15arcsec_fcube_cor.fits', 'abell141.png')
fits_to_png('Abell_209_aFix_pol_I_15arcsec_fcube_cor.fits', 'abell209.png')
fits_to_png('Abell_S295_aFix_pol_I_15arcsec_fcube_cor.fits', 'abells295.png')


FileNotFoundError: [Errno 2] No such file or directory: 'Source_Detection/Abell_141_aFix_pol_I_15arcsec_fcube_cor.fits'